# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la passe,45.5549,-0.8967,16.05,81,6,2.74,FR,1724205608
1,1,constantia,44.1833,28.6500,20.39,75,20,3.50,RO,1724205609
2,2,callao,-12.0667,-77.1500,17.62,82,100,5.14,PE,1724205378
3,3,atafu village,-8.5421,-172.5159,27.84,78,100,11.07,TK,1724205611
4,4,yanam,16.7333,82.2167,27.33,91,100,1.52,IN,1724205612


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
cleaned_df = city_data_df.loc[(city_data_df['Max Temp']<=25)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(cleaned_df))

# Drop any rows with null values
cleaned_df.dropna()

# Display sample data
cleaned_df.head()

55


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,adamstown,-25.0660,-130.1015,17.72,63,77,9.47,PN,1724205623
26,26,ipu,-4.3222,-40.7108,21.86,51,0,0.42,BR,1724205636
32,32,afaahiti,-17.7500,-149.2833,23.22,66,17,12.82,PF,1724205645
42,42,tolanaro,-25.0319,46.9987,19.04,68,40,5.14,MG,1724205658
78,78,yellowknife,62.4560,-114.3525,21.90,69,75,1.03,CA,1724205687


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,adamstown,PN,-25.0660,-130.1015,63,
26,ipu,BR,-4.3222,-40.7108,51,
32,afaahiti,PF,-17.7500,-149.2833,66,
42,tolanaro,MG,-25.0319,46.9987,68,
78,yellowknife,CA,62.4560,-114.3525,69,
83,canico,PT,32.6333,-16.8500,69,
110,iqaluit,CA,63.7506,-68.5145,61,
133,morondava,MG,-20.2833,44.2833,56,
148,queenstown,ZA,-31.8976,26.8753,24,
150,williams lake,CA,52.1415,-122.1445,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1
   }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
ipu - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
tolanaro - nearest hotel: Hôtel Mahavokey
yellowknife - nearest hotel: No hotel found
canico - nearest hotel: Dom Pedro
iqaluit - nearest hotel: Frobisher Inn
morondava - nearest hotel: Hotel Sharon Menabe
queenstown - nearest hotel: Queens Hotel
williams lake - nearest hotel: Coast Fraser Inn
saint-mandrier-sur-mer - nearest hotel: Le Barock
bayramaly - nearest hotel: No hotel found
balqash - nearest hotel: No hotel found
tadine - nearest hotel: Hôtel Nengoné
richards bay - nearest hotel: SUN1 Richards Bay
khovd - nearest hotel: Khovd hotel
glodeni - nearest hotel: Șarm Hotel
imishli - nearest hotel: No hotel found
whakatane - nearest hotel: Whakatane Hotel
amos - nearest hotel: Hôtel des Eskers
centenario do sul - nearest hotel: No hotel found
qulsary - nearest hotel: Камилла
khorixas - nearest hotel: iGowati Country Hotel and Lodge
blenheim - near

,City,Country,Lat,Lng,Humidity,Hotel Name
14,adamstown,PN,-25.0660,-130.1015,63,No hotel found
26,ipu,BR,-4.3222,-40.7108,51,No hotel found
32,afaahiti,PF,-17.7500,-149.2833,66,Omati Lodge
42,tolanaro,MG,-25.0319,46.9987,68,Hôtel Mahavokey
78,yellowknife,CA,62.4560,-114.3525,69,No hotel found
83,canico,PT,32.6333,-16.8500,69,Dom Pedro
110,iqaluit,CA,63.7506,-68.5145,61,Frobisher Inn
133,morondava,MG,-20.2833,44.2833,56,Hotel Sharon Menabe
148,queenstown,ZA,-31.8976,26.8753,24,Queens Hotel
150,williams lake,CA,52.1415,-122.1445,40,Coast Fraser Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)